In [3]:
import nltk.downloader

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import pandas as pd

In [7]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [8]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [9]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [12]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [13]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [15]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [16]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [17]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


CPU times: user 21 s, sys: 916 ms, total: 22 s
Wall time: 22 s


In [18]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 3min 21s, sys: 185 ms, total: 3min 22s
Wall time: 3min 22s


In [19]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [20]:
common_dictionary[10]

'ватутин'

In [21]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 42.1 s, sys: 20.6 s, total: 1min 2s
Wall time: 40 s


In [22]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(2, 0.23330995), (4, 0.36142796), (7, 0.34970027), (9, 0.035073977)]

In [23]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: военный сша американский космический запуск армия технология
topic_1: учёный вода район остров метр статья поиск
topic_2: журнал университет хороший солнце большой первый nn
topic_3: ракета рост млн объём составить выплата тыс
topic_4: гражданин журнал космос форум фронт народный городской
topic_5: погибнуть миссия кожа операция nn предупредить пассажир
topic_6: исследователь население ребёнок пострадать крым день район
topic_7: всё земля первый очень жизнь тело большой
topic_8: турист рак способность граница иран русский туристический
topic_9: мозг всё температура автор пенсия обнаружить способ
topic_10: самолёт источник корабль земля данные эксперимент экипаж
topic_11: луна век клетка поверхность учёный nn новый
topic_12: россия развитие новый рынок научный российский проект
topic_13: доллар экономика депутат штат налог жертва тело
topic_14: смерть проверка астероид планета астроном северный погода
topic_15: взрыв следствие телескоп украина песок пресссекретарить путин
topic

In [24]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [25]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.078009,0.128746,0.030664,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.129207,0.047848,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.570028
1,4896,0.0,0.0,0.066648,0.000000,0.000000,0.0,0.0,0.128966,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.782922,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,4897,0.0,0.0,0.233470,0.000000,0.361432,0.0,0.0,0.349537,0.0,0.035072,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,4898,0.0,0.0,0.716187,0.000000,0.000000,0.0,0.0,0.214900,0.0,0.000000,0.0,0.0,0.058437,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,4899,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.974054,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [26]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [27]:
doc_dict[293622]

array([0.        , 0.0214393 , 0.        , 0.        , 0.02087479,
       0.        , 0.22581194, 0.        , 0.11955556, 0.01742761,
       0.        , 0.10877936, 0.06238631, 0.        , 0.        ,
       0.0397971 , 0.13762569, 0.0571178 , 0.02027267, 0.        ,
       0.        , 0.        , 0.        , 0.13827708, 0.02547345])

In [28]:
user_articles_list = users['articles'].iloc[33]

### Домашнее задание

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import itertools

%matplotlib inline

In [36]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


Передалаем функцию и добавим в нее параметр выбора функции для обработки(mean, median, etc)

In [37]:
def get_user_embedding(user_articles_list, func = np.mean):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, 0)
    return user_vector

Сделаем вспомогательную функцию, для получения метрик, для разных функций запуска

In [51]:
def get_model_metrics(func=np.mean):
  user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, func), 1)])
  user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
  user_embeddings['uid'] = users['uid'].values
  user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
  X = pd.merge(user_embeddings, target, 'left')
  X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
  logreg = LogisticRegression()
  logreg.fit(X_train, y_train)

  preds = logreg.predict_proba(X_test)[:, 1]
  precision, recall, thresholds = precision_recall_curve(y_test, preds)
  fscore = (2 * precision * recall) / (precision + recall)
  # locate the index of the largest f score
  ix = np.argmax(fscore)
  roc_auc = roc_auc_score(y_test, preds)
  return thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc

  

In [52]:
all_metrics = []

#### Получим результаты для разных моделей

In [53]:
all_metrics.append(get_model_metrics(func=np.mean))
all_metrics.append(get_model_metrics(func=np.median))
all_metrics.append(get_model_metrics(func=np.max))

In [54]:
result = pd.DataFrame(all_metrics, columns=['threshold', 'F-Score', 'Precision', 'Recall', 'ROC_AUC'], index=['Mean','Median','Max'])
result

,threshold,F-Score,Precision,Recall,ROC_AUC
Mean,0.216622,0.559603,0.470752,0.689796,0.902027
Median,0.276677,0.688391,0.686992,0.689796,0.954339
Max,0.284486,0.757042,0.665635,0.877551,0.970624
